### <font color="orange" ><font size="4">Wellbore Two-Phase Flow Simulation in Geothermal System (Up to Down)</font></font>

<font size="2">This script tries to conduct simple wellbore simulation in two-phase flow simulate well condition. The two phase flow condition use Homogeneous method. Data used in this calculation is synthetic data. The assumption for this simple calculation is the casing only one means only use one roughness, and one type of casing diameter.

The Homogeneous method for calculating the pressure drop in two-phase flow involves applying the Poiseuille flow equation while using calculated variables for the two-phase fluid. These variables include the dynamic viscosity of the two-phase fluid, the density of the two-phase fluid, the Reynolds number for the two-phase fluid, and the friction factor for the two-phase flow.</font>


In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from scipy.optimize import brentq

from pyXSteam.XSteam import XSteam
steam_table = XSteam(XSteam.UNIT_SYSTEM_MKS) # Use SI units

<font size="2">
The equation is shown by following:

For two phase density and dynamic viscosity, using the harmonic weighting of liquid and steam properties

$$ \rho_{TP} = \frac{1}{(\frac{x}{\rho_g}+\frac{(1-x)}{\rho_l})}$$

$$ \mu_{TP} = x\mu_g + (1-x)\mu_l$$

For the Reynold number, the calculation used the recent two properties. Where the two phase velocity is also calculated from two phase density

$$ Re_{TP}=\frac{\rho_{TP} D v_{TP}}{\mu_{TP}}$$
where the velocity
$$ v_{TP}=\frac{\dot{m_t}}{\rho_{TP}\times A}$$


The friction factor is calculated using colebrook equation
$$ \frac{1}{\sqrt{f}} = -2.0 \log(\frac{\frac{\varepsilon}{D}}{3.7} + \frac{2.51}{Re\sqrt{f}}) $$

Lastly, the pressure drop is calculated by the friction pressure drop and also by the gravity pressure drop
$$ \frac{\Delta P_{TP}}{dy}= f\frac{ \rho_{TP} \, v^2_{TP}}{2 \, D}$$
</font>

In [2]:
#Defining the function to calculate the pressure drop
# Calculation definition
def dryness_fraction(pressure, enthalpy):
    h_L = steam_table.hL_p(pressure)
    h_V = steam_table.hV_p(pressure)
    dryness = (enthalpy-h_L)/(h_V - h_L)
    return dryness

def mix_density(pressure, dryness_frac):
    rho_L = steam_table.rhoL_p(pressure)
    rho_V = steam_table.rhoV_p(pressure)
    rho_mix = 1/((dryness_frac/rho_V)+((1-dryness_frac)/rho_L))
    return rho_mix

def mix_dyna_visc(pressure, dryness_frac):
    mu_L = steam_table.my_ph(pressure, steam_table.hL_p(pressure))
    mu_V = steam_table.my_ph(pressure, steam_table.hV_p(pressure))
    # mu_mix = 1/((dryness_frac/mu_V)+((1-dryness_frac)/mu_L))
    mu_mix = ((dryness_frac*mu_V)+((1-dryness_frac)*mu_L))
    return mu_mix

def mix_velocity(mass_flow, mix_density, pipe_diam):
    velocity_TP = mass_flow/(mix_density*np.pi*(pipe_diam**2)/4)
    return velocity_TP

def mix_reynold_num(mix_density, pipe_diam, mix_velocity, mix_dyna_visc):
    Re_mix = (mix_density*pipe_diam*mix_velocity)/mix_dyna_visc
    return Re_mix

def colebrook(f, rel_rough, reynolds_num):
    return 1/np.sqrt(f) + 2*np.log10(rel_rough/3.7 + 2.51/(reynolds_num*np.sqrt(f)))
def solve_colebrook(rel_roughness, reynolds_num):
    return brentq(colebrook, 0.005, 0.1, args=(rel_roughness, reynolds_num))

def fric_press_drop(fric_factor, diameter, mix_density, mix_velocity):
    Delta_P_fric = fric_factor*mix_density*np.power(mix_velocity,2)/(2*diameter)
    return Delta_P_fric

def grav_press_drop(mix_density, theta):
    Delta_P_grav = (mix_density*9.81*np.sin(theta))
    return Delta_P_grav

In [ ]:
# The method is Up to Down, use the Well Head Pressure as the initial pressure
# Well head pressure
Pwh =  13.5 # bar a
# Total mass flow rate (measured in the surface)
mass_flow = 26 # kg/s 
# Enthalpy of the two phase fluid
enthalpy_mix = 1500 # kJ/kg

# Pipe data
# How much depth the pipe is going down each iteration
depth_increment = 100 # m
# Casing length
casing_depth = 1500 # m
# Casing internal diameter
casing_ID = 0.317881 # m 
# Casing roughnes
roughness = 4.57e-5
# Casing inclination
inclination = 90 # degree means vertical well
theta = np.radians(inclination)
# relative roughness
rel_rough = roughness/casing_ID

In [4]:
# Creating Lists for csv file
list_depth = np.array([])
list_press_in = np.array([])
list_press_fin = np.array([])
list_dryness = np.array([])
list_friction = np.array([])
list_gravity = np.array([])

In [5]:
## Pressure drop calculation
press_it = Pwh
depth = 0 #for iteration
# Iteration
while True:
    list_press_in = np.append(list_press_in, press_it)
    
    depth += depth_increment
    list_depth = np.append(list_depth, depth)
    press_it = press_it

    #Calculation
    x_frac = dryness_fraction(press_it, enthalpy_mix)
    rho_mix = mix_density(press_it, x_frac)
    mu_mix = mix_dyna_visc(press_it, x_frac)
    v_mix = mix_velocity(mass_flow, rho_mix, casing_ID)
    Re_mix = mix_reynold_num(rho_mix, casing_ID, v_mix, mu_mix)
    frict = solve_colebrook(rel_rough, Re_mix)
    dP_fric = fric_press_drop(frict, casing_ID, rho_mix, v_mix)
    dP_grav = grav_press_drop(rho_mix, theta)
    dP_total = (dP_fric*depth_increment + dP_grav*depth_increment)/1e5
    press_it = press_it + dP_total

    #Appending the results to the list
    list_press_fin = np.append(list_press_fin, press_it)
    list_friction = np.append(list_friction, dP_fric)
    list_gravity = np.append(list_gravity/10e5, dP_grav)
    list_dryness= np.append(list_dryness/10e5, x_frac)
    print(f"Pressure at {depth} m is {press_it} bar a")

    if depth >= casing_depth:
        break
    else:
        continue

Pressure at 100 m is 13.812496681302578 bar a
Pressure at 200 m is 14.126996728969862 bar a
Pressure at 300 m is 14.443691898455887 bar a
Pressure at 400 m is 14.762769135613858 bar a
Pressure at 500 m is 15.084411437827086 bar a
Pressure at 600 m is 15.408798625158951 bar a
Pressure at 700 m is 15.73610803585797 bar a
Pressure at 800 m is 16.066515158182803 bar a
Pressure at 900 m is 16.400194208599334 bar a
Pressure at 1000 m is 16.737318664851408 bar a
Pressure at 1100 m is 17.07806176114422 bar a
Pressure at 1200 m is 17.422596951647613 bar a
Pressure at 1300 m is 17.77109834768052 bar a
Pressure at 1400 m is 18.12374113324279 bar a
Pressure at 1500 m is 18.48070196298867 bar a


In [ ]:
# Saving output to CSV
data_dict = {
    'Column1': list_depth,
    'Column2': list_press_in,
    'Column3': list_dryness,
    'Column4': list_friction,
    'Column5': list_gravity,
    'Column6': list_press_fin,
}
df = pd.DataFrame(data_dict)
df.columns = ['Depth (m)','Pressure initial (bar a)','Dryness', 'dP due friction (Pa/m)', 'dP due gravity (Pa/m)', 'Pressure (bar a)']
df.to_csv('simple_wellbore_modelling.csv')

print("Successfully write the .csv file")

Successfully write the .csv file


<font size="2">Based on the wellbore two-phase flow model, when parameters such as the depth of the casing (1500 m), the internal casing diameter (0.317881 m), the wellhead pressure (Pwh, 13.5 bar a), and the mass flow rate (26 kg/s) are given, it shows that the flowing bottomhole pressure (Pwf) is 18.4807 bar a.  </font>